# Random Forest × IRT Study

This notebook walks through data preparation, model training, and Item Response Theory analysis for the CIFAR-10 subset.

## 0. Setup

Import libraries, define configuration, and set deterministic seeds for reproducibility.

In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

DATA_ROOT = Path('../data')
CACHE_DIR = DATA_ROOT
FIGURES_DIR = Path('../figures')
MODELS_DIR = Path('../models')
SUBSET_ARCHIVE = CACHE_DIR / 'cifar10_subset.npz'
EMBEDDINGS_ARCHIVE = CACHE_DIR / 'cifar10_embeddings.npz'
SEED = 42

np.random.seed(SEED)

# TODO: when executing, ensure directories exist before writing outputs.
# FIGURES_DIR.mkdir(parents=True, exist_ok=True)
# MODELS_DIR.mkdir(parents=True, exist_ok=True)

## 1. Data Download & Subsampling

Use the helper routines in `src.data_pipeline` to ensure CIFAR-10 is downloaded and stratified into manageable train/val/test splits.

In [ ]:
from src.data_pipeline import SubsetConfig, save_cifar10_subset

subset_config = SubsetConfig(data_root=CACHE_DIR)
if not SUBSET_ARCHIVE.exists():
    subset_archive = save_cifar10_subset(subset_config)
else:
    subset_archive = SUBSET_ARCHIVE
subset_archive

## 2. Embedding Pipeline

Flatten the cached tensors and project to a compact latent space with PCA to serve as Random Forest inputs.

In [ ]:
from src.data_pipeline import compute_pca_embeddings

if not EMBEDDINGS_ARCHIVE.exists():
    embeddings_path, embedding_summary = compute_pca_embeddings(subset_archive)
else:
    embeddings_path, embedding_summary = EMBEDDINGS_ARCHIVE, {
        'train_embeddings': None,
        'val_embeddings': None,
        'test_embeddings': None,
        'explained_variance_ratio': None,
    }
embedding_summary

## 3. Random Forest Training

Train a baseline `RandomForestClassifier` on the PCA embeddings and capture core metrics.

In [ ]:
import numpy as np
from pathlib import Path

embeddings = np.load(embeddings_path)
X_train = embeddings['train_embeddings']
X_val = embeddings['val_embeddings']
X_test = embeddings['test_embeddings']
y_train = embeddings['y_train']
y_val = embeddings['y_val']
y_test = embeddings['y_test']

# TODO (execution phase): optionally standardize embeddings.
# if not embeddings.get('whitened', False):
#     scaler = StandardScaler().fit(X_train)
#     X_train = scaler.transform(X_train)
#     X_val = scaler.transform(X_val)
#     X_test = scaler.transform(X_test)
#     np.savez(CACHE_DIR / 'embedding_scaler.npz', mean_=scaler.mean_, scale_=scaler.scale_)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=SEED,
    oob_score=True,
)

# Execution checklist (leave commented until ready):
# 1. rf.fit(X_train, y_train)
# 2. y_pred = rf.predict(X_test)
# 3. probas = rf.predict_proba(X_test)
# 4. metrics = {
#        'overall_accuracy': float(accuracy_score(y_test, y_pred)),
#        'oob_accuracy': float(rf.oob_score_),
#        'val_accuracy': float(accuracy_score(y_val, rf.predict(X_val))),
#        'per_class_accuracy': accuracy_score(y_test, y_pred, normalize=False)  # adjust to per-class table
#    }
# 5. conf_mat = confusion_matrix(y_test, y_pred)
# 6. permutation = permutation_importance(
#        rf, X_val, y_val, n_repeats=10, random_state=SEED, n_jobs=-1
#    )
# 7. persist metrics/arrays to CACHE_DIR / 'rf_metrics.json' and 'rf_confusion.npy'.
# 8. dump feature importances & permutation results for slide consumption.

# Suggested persistence stub:
# metrics_payload = {
#     'metrics': metrics,
#     'classes': embeddings['classes'].tolist(),
#     'feature_importances': rf.feature_importances_.tolist(),
# }
# (CACHE_DIR / 'rf_metrics.json').write_text(json.dumps(metrics_payload, indent=2))
# np.save(CACHE_DIR / 'rf_confusion.npy', conf_mat)
# permutation_df = pd.DataFrame(
#     {'feature': np.arange(permutation.importances_mean.size),
#      'importances_mean': permutation.importances_mean,
#      'importances_std': permutation.importances_std}
# )
# permutation_df.to_csv(CACHE_DIR / 'rf_permutation_importance.csv', index=False)

# Visualization hook (to be run later):
# import seaborn as sns
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(6, 5))
# sns.heatmap(conf_mat / conf_mat.sum(axis=1, keepdims=True), annot=True, fmt='.2f', ax=ax)
# ax.set_title('Random Forest Confusion Matrix (Normalized)')
# ax.set_xlabel('Predicted')
# ax.set_ylabel('True')
# fig.tight_layout()
# fig_path = FIGURES_DIR / 'rf_confusion_matrix.png'
# FIGURES_DIR.mkdir(parents=True, exist_ok=True)
# fig.savefig(fig_path, dpi=200)
# plt.close(fig)


## 4. Response Matrix Construction

Collect per-tree predictions on the test split to assemble the binary response matrix `R`.

In [ ]:
import numpy as np

# Response matrix utility --------------------------------------------------
# TODO: ensure RF is trained before running this section.
# response_matrix shape: (n_trees, n_test_examples)
# Each entry is 1 if estimator predicts correctly, else 0.

def build_response_matrix(rf_model, X, y_true):
    """Return binary matrix of estimator correctness for each test example."""
    responses = []
    for estimator in rf_model.estimators_:
        preds = estimator.predict(X)
        responses.append((preds == y_true).astype(np.uint8))
    return np.stack(responses)

# Execution plan (commented for now):
# R = build_response_matrix(rf, X_test, y_test)
# response_path = CACHE_DIR / 'response_matrix.npz'
# np.savez(response_path, R=R, y_test=y_test, classes=embeddings['classes'])
# Wright-map compatible summary stats:
# item_accuracy = R.mean(axis=0)
# tree_accuracy = R.mean(axis=1)
# summary = {
#     'item_accuracy': item_accuracy.tolist(),
#     'tree_accuracy': tree_accuracy.tolist(),
# }
# (CACHE_DIR / 'response_summary.json').write_text(json.dumps(summary, indent=2))


## 5. IRT Fitting

Fit a Rasch (1PL) model using the response matrix to estimate tree ability (θ) and item difficulty (δ).

In [ ]:
# IRT fitting checklist ----------------------------------------------------
# Preferred library: py_irt (fallback: pyirt). Expect binary matrix input.
# Example workflow once R is available:
# from py_irt.irt import irt_1pl
# model = irt_1pl(R)
# tree_ability = model['theta']           # shape (n_trees,)
# item_difficulty = model['delta']        # shape (n_items,)
# discrimination = model.get('a')         # optional for 2PL
# log_likelihood = model.get('loglik')
#
# Diagnostics to capture:
# - Convergence flag / iterations
# - Mean/std of θ and δ
# - Histogram-ready arrays saved to CACHE_DIR / 'irt_parameters.npz'
#
# Suggested persistence stub:
# np.savez(
#     CACHE_DIR / 'irt_parameters.npz',
#     theta=tree_ability,
#     delta=item_difficulty,
#     a=discrimination,
#     meta={'loglik': log_likelihood}
# )
# Additionally, emit JSON summary for slides/reporting.


## 6. Comparative Analysis

Contrast IRT parameters with Random Forest margins, feature importances, and error patterns.

In [ ]:
# TODO: compute correlations, Wright map visualization, and hard example list

## 7. Slide Export

Append generated plots and key findings to `slides.md`.

In [ ]:
# TODO: once RF + IRT outputs are ready, assemble comparison plots.
# Suggested steps:
# 1. Compute RF margin per example from `probas` gathered above (p_true - max_other).
#    margins = probas[np.arange(len(y_test)), y_test] - np.max(np.where(one_hot_indices != y_test[:, None], probas, -np.inf), axis=1)
# 2. Build a dataframe with columns ['delta', 'margin', 'entropy', 'class', 'hard_example_flag'].
# 3. Correlate item difficulty δ with margin, entropy, per-class error, and RF confidence.
#    store both Pearson and Spearman correlations in CACHE_DIR / 'correlations.json'.
# 4. Produce Wright map with matplotlib/seaborn; save to FIGURES_DIR / 'wright_map.png'.
#    Example: twin histogram or scatter aligning θ (trees) on top axis and δ (items) on bottom axis.
# 5. Surface top-10 hardest items (largest δ) along with thumbnails (use original tensors).
#    Persist indices to CACHE_DIR / 'hard_items.json' and copy images to FIGURES_DIR / hard_examples/.
# 6. Summarize findings in dictionaries to feed slides export and README logs.
# 7. Persist correlation table to CSV for reproducibility; include ranking of trees/items by ability/difficulty.


## New analytics: RF margins, δ correlations, visuals

Run helper scripts to compute RF per-item signals, correlate them with IRT difficulty, and emit supporting figures (difficulty vs margin/entropy, Wright map, hardest/easiest montages, class difficulty summary).

In [ ]:
import subprocess
import sys
from pathlib import Path

project_root = Path.cwd().parent
scripts = project_root / "scripts"

commands = [
    [sys.executable, str(scripts / "compute_rf_signals.py")],
    [sys.executable, str(scripts / "analyze_rf_irt_correlations.py")],
    [sys.executable, str(scripts / "plot_wright_map.py")],
    [sys.executable, str(scripts / "visualize_difficulty_extremes.py"), "--split", "test", "--count", "10"],
    [sys.executable, str(scripts / "class_difficulty_summary.py")],
]

for cmd in commands:
    print("Running", " ".join(cmd))
    completed = subprocess.run(cmd, cwd=project_root)
    if completed.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}")
print("All analytics scripts completed.")